In [1]:
import requests,json,os,time
import random
apiKey=os.getenv('CKANAPIKEY')
if apiKey is None:
    raise EnvironmentError("Failed because {} is not set.".format('CKANAPIKEY'))

In [2]:
def getFieldsTemplate():
    template = {
                "id": None,
                "info": 
                {
                    "label": None,
                    "notes": None
                }
            }
    return template

def getFieldsForCKAN(metadata):
    fieldsList=[]
    for key,val in metadata.items():
        template = getFieldsTemplate()
        template['id'] = key
        template['info']['label'] = key
        template['info']['notes'] = val
        fieldsList.append(template)
    return fieldsList

In [3]:
mainDir = '/Users/iperezx/Documents/sage-commons/sage-commons/'
sageCommonsURL='http://127.0.0.1:5000'
headers = {'Authorization': apiKey}
orgName= 'sage'

## import sdr data with query

### create data resource

In [4]:
apiAction='/api/3/action/package_create'
url = sageCommonsURL+apiAction
title = 'metsense' + str(random.randint(1, 1e4))

name = title
# spatialVal = {
#   "type":"Polygon",
#   "coordinates":[[[2.05827, 49.8625],[2.05827, 55.7447], [-6.41736, 55.7447], [-6.41736, 49.8625], [2.05827, 49.8625]]]
# }
#extras = {'key': "spatial",'value': json.dumps(spatialVal) }
tags = [{'name':'json'},{'name':'metsense'}]

payload = {'owner_org': orgName,
           'title': title,
           'name' : name,
           'notes': 'Description of dataset',
           'tags' : tags,
           #'extras': [extras]
           
          }

response = requests.request("POST", url, headers=headers, json = payload)
jsonResponseDS = response.json()
print(response)
print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

<Response [200]>
{
    "help": "http://docker.for.mac.localhost:5000/api/3/action/help_show?name=package_create",
    "result": {
        "author": null,
        "author_email": null,
        "creator_user_id": "fee53ee6-064e-40c9-ad3f-0d9ad6dd1bda",
        "extras": [],
        "groups": [],
        "id": "aa35faf8-b199-4078-8565-ef6e0a70192f",
        "isopen": false,
        "license_id": null,
        "license_title": null,
        "maintainer": null,
        "maintainer_email": null,
        "metadata_created": "2021-03-17T23:44:09.458652",
        "metadata_modified": "2021-03-17T23:44:09.458666",
        "name": "metsense3947",
        "notes": "Description of dataset",
        "num_resources": 0,
        "num_tags": 2,
        "organization": {
            "approval_status": "approved",
            "created": "2021-01-28T02:29:24.314230",
            "description": "",
            "id": "26aa9160-b30d-42d4-b879-2b7b90b0dcc5",
            "image_url": "",
            "is_organi

### create data source

In [5]:
apiAction='/api/3/action/sagecommons_create'
packageID = jsonResponseDS['result']['id']
url = sageCommonsURL+apiAction

resourceName = 'plugin-metsense'
resourceDescription = 'Metsense detection plugin inference data'
dataURL ='https://sdr.honeyhouse.one/api/v1/query'
#dataURL = 'http://service-ip:10000/api/v1/query'
query = {"start": "-10s",
        "filter": {
            "plugin": "metsense:.*"
        }
}

resource = {'package_id': packageID,
            'name' : resourceName,
            'description': resourceDescription,
            'url': dataURL,
            'format':'JSON',
            'query':query#query not yet supporting this
           }


payload = {'resource': resource,
          }

start = time.time() # start timing
response = requests.request("POST", url, headers=headers, json = payload)
end = time.time()
elapseTime = abs(end-start)
print('Elapsed time: ' + str(elapseTime))

Elapsed time: 0.1374530792236328


In [6]:
jsonResponse = response.json()
print(response)
print(response.text)
print(json.dumps(jsonResponse, indent=4, sort_keys=True))

<Response [200]>
{"help": "http://docker.for.mac.localhost:5000/api/3/action/help_show?name=sagecommons_create", "success": true, "result": {"resource": {"mimetype": null, "name": "plugin-metsense", "format": "JSON", "url": "https://sdr.honeyhouse.one/api/v1/query", "package_id": "aa35faf8-b199-4078-8565-ef6e0a70192f", "query": {"filter": {"plugin": "metsense:.*"}, "start": "-10s"}, "description": "Metsense detection plugin inference data"}, "resource_id": "2f25cd83-29a6-4127-b963-c61ceca907af"}}
{
    "help": "http://docker.for.mac.localhost:5000/api/3/action/help_show?name=sagecommons_create",
    "result": {
        "resource": {
            "description": "Metsense detection plugin inference data",
            "format": "JSON",
            "mimetype": null,
            "name": "plugin-metsense",
            "package_id": "aa35faf8-b199-4078-8565-ef6e0a70192f",
            "query": {
                "filter": {
                    "plugin": "metsense:.*"
                },
         

In [7]:
apiAction='/api/3/action/resource_view_list'
resourceID = jsonResponse['result']['resource_id']
url = sageCommonsURL+apiAction


payload = {'id': resourceID,
          }

response = requests.request("GET", url, headers=headers, json = payload)
jsonResponse = response.json()
print(json.dumps(jsonResponse, indent=4, sort_keys=True))

{
    "help": "http://docker.for.mac.localhost:5000/api/3/action/help_show?name=resource_view_list",
    "result": [
        {
            "description": "",
            "id": "338d8432-cac5-401b-a950-efea6c587c87",
            "package_id": "aa35faf8-b199-4078-8565-ef6e0a70192f",
            "resource_id": "2f25cd83-29a6-4127-b963-c61ceca907af",
            "title": "Text",
            "view_type": "text_view"
        },
        {
            "description": null,
            "id": "276fac51-a1d8-4122-b43d-68bf4a061289",
            "package_id": "aa35faf8-b199-4078-8565-ef6e0a70192f",
            "resource_id": "2f25cd83-29a6-4127-b963-c61ceca907af",
            "title": "Sage Interface View",
            "view_type": "sageinterfaceview"
        }
    ],
    "success": true
}


In [8]:
jsonResponse['result'][0]['id']

'338d8432-cac5-401b-a950-efea6c587c87'

In [9]:
apiAction='/api/3/action/resource_view_delete'
viewID = jsonResponse['result'][0]['id']
url = sageCommonsURL+apiAction


payload = {'id': viewID,
          }

response = requests.request("POST", url, headers=headers, json = payload)
jsonResponse = response.json()
print(json.dumps(jsonResponse, indent=4, sort_keys=True))

{
    "help": "http://docker.for.mac.localhost:5000/api/3/action/help_show?name=resource_view_delete",
    "result": null,
    "success": true
}


## Vocabulary


### create vocabulary

In [ ]:
apiAction='/api/3/action/vocabulary_create'
url = sageCommonsURL+apiAction

name = 'plugin'

tags = [{'name':'smoke_detection'},{'name':'cloud_detection'},
        {'name':'x_detection'},{'name':'y_detection'},
        {'name':'z_detection'}]

payload = {
           'name' : name,
           'tags' : tags
          }

response = requests.request("POST", url, headers=headers, json = payload)
jsonResponseDS = response.json()
print(response)
print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

### Get vocabulary

In [ ]:
apiAction='/api/3/action/vocabulary_list'
url = sageCommonsURL+apiAction

response = requests.request("GET", url, headers=headers, json = payload)
jsonResponseDS = response.json()
print(response)
print(json.dumps(jsonResponseDS, indent=4, sort_keys=True))

In [ ]:
import json
import requests
url = "https://sdr.honeyhouse.one/api/v1/query"
payload={"start":"-10s"}
response = requests.post(url, data=json.dumps(payload),verify=False)
print(response.text)